In [1]:
import pandas as pd
import numpy as np
import sweetviz as sv
from pydataset import data

In [2]:
mpg = data('mpg')

In [3]:
# advert_report = sv.analyze(mpg)

In [4]:
# advert_report.show_html('Advertising.html')

In [5]:
mpg

,manufacturer,model,displ,year,cyl,trans,drv,cty,hwy,fl,class
1,audi,a4,1.8,1999,4,auto(l5),f,18,29,p,compact
2,audi,a4,1.8,1999,4,manual(m5),f,21,29,p,compact
3,audi,a4,2.0,2008,4,manual(m6),f,20,31,p,compact
4,audi,a4,2.0,2008,4,auto(av),f,21,30,p,compact
5,audi,a4,2.8,1999,6,auto(l5),f,16,26,p,compact
...,...,...,...,...,...,...,...,...,...,...,...
230,volkswagen,passat,2.0,2008,4,auto(s6),f,19,28,p,midsize
231,volkswagen,passat,2.0,2008,4,manual(m6),f,21,29,p,midsize
232,volkswagen,passat,2.8,1999,6,auto(l5),f,16,26,p,midsize
233,volkswagen,passat,2.8,1999,6,manual(m5),f,18,26,p,midsize


In [6]:
flu = pd.read_csv('Flu_Shot_Learning_Predict_H1N1_and_Seasonal_Flu_Vaccines_-_Training_Features.csv')

In [7]:
# flu_report = sv.analyze(flu)
# flu_report.show_html('flu.html')

In [8]:
def get_db_url(dbname) -> str:
    url = 'mysql+pymysql://{}:{}@{}/{}'
    return url.format(user, password, host, dbname)


def get_telco_data():
    query = '''
    SELECT customers.customer_id, gender, senior_citizen, 
    partner, dependents, tenure, monthly_charges, 
    total_charges, phone_service, multiple_lines, 
    online_security, online_backup, device_protection, 
    tech_support, streaming_tv, streaming_movies, 
    paperless_billing, customers.contract_type_id, 
    customers.payment_type_id, customers.internet_service_type_id, 
    churn
FROM customers
LEFT JOIN contract_types ON customers.contract_type_id=contract_types.contract_type_id
LEFT JOIN internet_service_types ON customers.internet_service_type_id=internet_service_types.internet_service_type_id
LEFT JOIN payment_types ON customers.payment_type_id=payment_types.payment_type_id
    '''
    df = pd.read_sql(query, get_db_url('telco_churn'))
    return df 

In [9]:
from env import user, password, host

In [10]:
df = get_telco_data()

In [11]:
df

,customer_id,gender,senior_citizen,partner,dependents,tenure,monthly_charges,total_charges,phone_service,multiple_lines,...,online_backup,device_protection,tech_support,streaming_tv,streaming_movies,paperless_billing,contract_type_id,payment_type_id,internet_service_type_id,churn
0,0002-ORFBO,Female,0,Yes,Yes,9,65.60,593.3,Yes,No,...,Yes,No,Yes,Yes,No,Yes,2,2,1,No
1,0003-MKNFE,Male,0,No,No,9,59.90,542.4,Yes,Yes,...,No,No,No,No,Yes,No,1,2,1,No
2,0004-TLHLJ,Male,0,No,No,4,73.90,280.85,Yes,No,...,No,Yes,No,No,No,Yes,1,1,2,Yes
3,0011-IGKFF,Male,1,Yes,No,13,98.00,1237.85,Yes,No,...,Yes,Yes,No,Yes,Yes,Yes,1,1,2,Yes
4,0013-EXCHZ,Female,1,Yes,No,3,83.90,267.4,Yes,No,...,No,No,Yes,Yes,No,Yes,1,2,2,Yes
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7038,9987-LUTYD,Female,0,No,No,13,55.15,742.9,Yes,No,...,No,No,Yes,No,No,No,2,2,1,No
7039,9992-RRAMN,Male,0,Yes,No,22,85.10,1873.7,Yes,Yes,...,No,No,No,No,Yes,Yes,1,1,2,Yes
7040,9992-UJOEL,Male,0,No,No,2,50.30,92.75,Yes,No,...,Yes,No,No,No,No,Yes,1,2,1,No
7041,9993-LHIEB,Male,0,Yes,Yes,67,67.85,4627.65,Yes,No,...,No,Yes,Yes,No,Yes,No,3,2,1,No


In [12]:
from sklearn.preprocessing import MinMaxScaler


def wrangle_telco(df):
    '''
    Transforms data brought in from SQL to handle nulls in tenure
    Turn the total_charges column to a float
    Add calculated fields
    '''
    df['automatic_payment'] = ((df['payment_type_id'] == 3) | (df['payment_type_id'] == 4))
    scaler = MinMaxScaler()
    df['monthly_charges_scaled'] = scaler.fit_transform(df['monthly_charges'].values.reshape(-1,1))
    df['tenure_3_or_less'] = df['tenure']<=3
    return df

In [13]:
df = wrangle_telco(df)

In [14]:
df.total_charges.isna().sum()

0

In [15]:
df.monthly_charges.isna().sum()

0

In [16]:
from pandas_profiling import ProfileReport

ModuleNotFoundError: No module named 'pandas_profiling'